In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math as math
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from datetime import date
from datetime import datetime


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import tree

from sklearn import ensemble

from collections import Counter, defaultdict
from itertools import combinations 
import operator
import math
import itertools
from sklearn.feature_extraction import DictVectorizer
from sklearn import preprocessing, tree

In [2]:
def agg_string_to_arr(arr):
    new_arr = []
    for e in arr:
        if e not in new_arr:
            new_arr.append(e)
    ## New
    # Sort list
    new_arr.sort()
    new_str = ';'.join(map(str, new_arr))
    return new_str

In [3]:
def attendance_demog_wash_by_year(attendance, demog):
    #CSV -> DF
    attendance_df = pd.read_csv(attendance)
    demog_df = pd.read_csv(demog)
    
    #ATTENDANCE
    #----------
    
    #Extrapolate attendance specifier as boolean columns
    one_hot_atten = pd.get_dummies(attendance_df['Attendance'])
    one_hot_sesh = pd.get_dummies(attendance_df['Session'])
    attendance_clean = attendance_df.join(one_hot_atten)
    attendance_clean = attendance_clean.join(one_hot_sesh)
    
    #Combine rows by ChildID and 
    attendance_clean = attendance_clean.groupby(['ChildID'],as_index=False).agg(
             {'Best Interest': 'sum',
              'In Attendance': 'sum',
              'Other': 'sum',
              'Sick':'sum',
              'Unexcused Absence':'sum',
              'FD':'sum',
              'PD':'sum',
              'Center':agg_string_to_arr})
    
    #Add aggregate columns: Total Attendance,Absence, and Days AND Attendance Rate
    attendance_clean['Total Attendance'] = attendance_clean['In Attendance']
    attendance_clean['Total Absence'] = attendance_clean['Other'] + attendance_clean['Sick'] + attendance_clean['Unexcused Absence'] + attendance_clean['Best Interest']
    attendance_clean['Total Days'] = attendance_clean['Total Attendance'] + attendance_clean['Total Absence']
    attendance_clean['Attendance Rate'] = attendance_clean['Total Attendance'] / attendance_clean['Total Days']
    
    attendance_clean_final = attendance_clean
    
    #DEMOGRAPHICS
    #------------
    
    #Reorder Columns
    demog_clean = demog_df[['ChildID','ChildEthnicity','ChildRace','ChildLanguage','ChildFlags','ChildDateofBirth']]
    
    #Drop NAN 
    demog_clean = demog_clean[demog_clean['ChildDateofBirth'].notna()]

    #Add current age column
    today = datetime.today() 

    def get_age(bday_str):
        datetime_bday = datetime.strptime(str(bday_str),"%m/%d/%Y")
        age = (today-datetime_bday).days // 365
        return age

    demog_clean['Age (as of current day)'] = demog_clean['ChildDateofBirth'].apply(get_age)
    demog_clean['ChildID'] = demog_clean.ChildID.astype(int)
    
    #Aggregate rows to combine duplicate ChildIDs
    
    #Df with only aggregated string columns
    combined_str = demog_clean.groupby('ChildID').agg({'ChildRace':agg_string_to_arr, 'ChildFlags':agg_string_to_arr})
    
    #Combine agg str with demog clean
    merged = combined_str.merge(demog_clean, on = 'ChildID',how = 'left')
    
    #Drop extraneous columns 
    cleaned = merged[['ChildID','ChildEthnicity','ChildLanguage','ChildRace_x','ChildFlags_x','ChildDateofBirth','Age (as of current day)']]
    
    #Drop duplicates rows
    demog_clean_final = cleaned.drop_duplicates(subset= 'ChildID')
    
    #Rename columns
    demog_clean_final = demog_clean_final.rename(columns = {"ChildRace_x":'ChildRace',"ChildFlags_x":'ChildFlags'})
    
    
    #JOIN DFs for MASTER
    master = pd.merge(demog_clean_final,attendance_clean_final, how='inner', on= 'ChildID')
    
    def attend_yes_no(dat):
        if dat > 0.7:
            return 1
        else:
            return 0
        
    master['Attend'] = master['Attendance Rate'].map(attend_yes_no)
    
    #EXPORT
#     #------
#     from pathlib import Path 

#     filepath = Path(attendance + '_out.csv')  
#     filepath.parent.mkdir(parents=True, exist_ok=True)  
#     master.to_csv(filepath)
    
    return master

df_1819 = attendance_demog_wash_by_year('18-19 attendance master.csv','PvH-demographicsdash_childdata.csv')
df_1819
df_1920 = attendance_demog_wash_by_year('19-20 attendance master.csv','PvH-demographicsdash_childdata.csv')
df_1920

,ChildID,ChildEthnicity,ChildLanguage,ChildRace,ChildFlags,ChildDateofBirth,Age (as of current day),Best Interest,In Attendance,Other,Sick,Unexcused Absence,FD,PD,Center,Total Attendance,Total Absence,Total Days,Attendance Rate,Attend
0,22717,Hispanic or Latino,01 -- Spanish,Caucasian,Infant/Toddler,01/13/2008,14,0,174,61,1,0,236.0,0,FH,174,62,236,0.737288,1
1,25173,Hispanic or Latino,00 -- English,Caucasian,Preschool,01/06/2007,15,9,157,61,9,0,236.0,0,FH,157,79,236,0.665254,0
2,30829,Hispanic or Latino,01 -- Spanish,Caucasian,Infant/Toddler,11/19/2010,12,0,174,61,1,0,236.0,0,FH,174,62,236,0.737288,1
3,31583,Hispanic or Latino,00 -- English,Caucasian,Emergency Care,05/22/2010,12,0,203,40,3,0,246.0,0,Linda Vista,203,43,246,0.825203,1
4,35649,Hispanic or Latino,00 -- English,Caucasian,Preschool,11/30/2007,15,0,27,0,8,0,35.0,0,Linda Vista,27,8,35,0.771429,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4898,68331,Hispanic or Latino,00 -- English,Caucasian,nan,10/11/2014,8,0,22,0,0,0,22.0,0,FH,22,0,22,1.000000,1
4899,68332,Hispanic or Latino,00 -- English,Caucasian,Admissions Agreement #14 (check box if Yes);Ad...,06/11/2016,6,0,22,0,0,0,22.0,0,FH,22,0,22,1.000000,1
4900,68333,Hispanic or Latino,00 -- English,Caucasian,Admissions Agreement #14 (check box if Yes);Ad...,07/07/2016,6,0,22,0,0,0,22.0,0,FH,22,0,22,1.000000,1
4901,68334,Hispanic or Latino,00 -- English,Caucasian,nan,05/09/2016,6,0,22,0,0,0,22.0,0,FH,22,0,22,1.000000,1


In [9]:
columns = [#'ChildID',
 'ChildEthnicity',
 'ChildLanguage',
 'ChildRace',
 'ChildFlags',
#  'ChildDateofBirth',
 'Age (as of current day)',
#  'Best Interest',
#  'In Attendance',
#  'Other',
#  'Sick',
#  'Unexcused Absence',
#  'FD',
#  'PD',
 'Center'#,
#  'Total Attendance',
#  'Total Absence',
#  'Total Days'#,
#  'Attendance Rate'
          ]

rows_base = df_1819[columns].to_dict(orient='row')
rows_1920 = df_1920[columns].to_dict(orient='row')

onehot_base = DictVectorizer(sparse=False).fit(rows_base)
onehot_1920 = DictVectorizer(sparse=False).fit(rows_1920)
X_base = onehot_base.transform(rows_base)
y_base = df_1819['Attend']#.as_matrix()

X_test = onehot_base.transform(rows_1920)
y_test = df_1920['Attend']

/var/folders/y8/s72vwprj7411g2sh4q078pvh0000gn/T/ipykernel_40200/3854362702.py:22: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  rows_base = df_1819[columns].to_dict(orient='row')
/var/folders/y8/s72vwprj7411g2sh4q078pvh0000gn/T/ipykernel_40200/3854362702.py:23: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  rows_1920 = df_1920[columns].to_dict(orient='row')


In [10]:
rows_base

[{'ChildEthnicity': 'Hispanic or Latino',
  'ChildLanguage': '01 -- Spanish',
  'ChildRace': 'Caucasian',
  'ChildFlags': 'Preschool',
  'Age (as of current day)': 17,
  'Center': 'FH'},
 {'ChildEthnicity': 'Hispanic or Latino',
  'ChildLanguage': '01 -- Spanish',
  'ChildRace': 'Caucasian',
  'ChildFlags': 'Infant/Toddler',
  'Age (as of current day)': 14,
  'Center': 'FH'},
 {'ChildEthnicity': 'Hispanic or Latino',
  'ChildLanguage': '00 -- English',
  'ChildRace': 'Caucasian',
  'ChildFlags': 'Preschool',
  'Age (as of current day)': 15,
  'Center': 'FH'},
 {'ChildEthnicity': 'Hispanic or Latino',
  'ChildLanguage': '01 -- Spanish',
  'ChildRace': 'Caucasian',
  'ChildFlags': 'Infant/Toddler',
  'Age (as of current day)': 12,
  'Center': 'FH'},
 {'ChildEthnicity': 'Hispanic or Latino',
  'ChildLanguage': '00 -- English',
  'ChildRace': 'Caucasian',
  'ChildFlags': 'Emergency Care',
  'Age (as of current day)': 12,
  'Center': 'Linda Vista'},
 {'ChildEthnicity': 'Hispanic or Latino',

In [11]:
rows_1920

[{'ChildEthnicity': 'Hispanic or Latino',
  'ChildLanguage': '01 -- Spanish',
  'ChildRace': 'Caucasian',
  'ChildFlags': 'Infant/Toddler',
  'Age (as of current day)': 14,
  'Center': 'FH'},
 {'ChildEthnicity': 'Hispanic or Latino',
  'ChildLanguage': '00 -- English',
  'ChildRace': 'Caucasian',
  'ChildFlags': 'Preschool',
  'Age (as of current day)': 15,
  'Center': 'FH'},
 {'ChildEthnicity': 'Hispanic or Latino',
  'ChildLanguage': '01 -- Spanish',
  'ChildRace': 'Caucasian',
  'ChildFlags': 'Infant/Toddler',
  'Age (as of current day)': 12,
  'Center': 'FH'},
 {'ChildEthnicity': 'Hispanic or Latino',
  'ChildLanguage': '00 -- English',
  'ChildRace': 'Caucasian',
  'ChildFlags': 'Emergency Care',
  'Age (as of current day)': 12,
  'Center': 'Linda Vista'},
 {'ChildEthnicity': 'Hispanic or Latino',
  'ChildLanguage': '00 -- English',
  'ChildRace': 'Caucasian',
  'ChildFlags': 'Preschool',
  'Age (as of current day)': 15,
  'Center': 'Linda Vista'},
 {'ChildEthnicity': 'Hispanic or

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_base, y_base, test_size=0.2, random_state=42
)
print(f'Training set size: {len(y_train)}')
print(f'Test set size: {len(y_test)}')

NameError: name 'X' is not defined

In [13]:
clf = LogisticRegression(max_iter=200)
clf.fit(X_base, y_base)
print(f'Test set accuracy: {clf.score(X_test, y_test)}')

Test set accuracy: 0.2765653681419539


/Users/hillaryjosephine/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
